In [26]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import Ridge, Lasso, ElasticNet

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Windows/Fonts/malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [27]:
df=pd.read_csv('2018 국민여행조사_국내여행.csv')

In [28]:
df['D_TRA1_SYEAR'].value_counts()

2018.0    14296
2017.0        7
Name: D_TRA1_SYEAR, dtype: int64

In [29]:
print(df['D_TRA1_SDAY'].isna().sum(), df['D_TRA1_COST'].isna().sum(), 
df['A1'].isnull().sum(), df['A1'].notnull().sum(), df['D_TRA1_CASE'].notnull().sum())

9851 9851 12306 11848 14303


In [30]:
# 여행 유형 미체크 항목을 0으로 채움
df['D_TRA1_CASE'].fillna(0, inplace=True)
df['D_TRA2_CASE'].fillna(0, inplace=True)
df['D_TRA3_CASE'].fillna(0, inplace=True)
df['D_TRA4_CASE'].fillna(0, inplace=True)

In [31]:
df['D_TRA1_CASE'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
24149    False
24150    False
24151    False
24152    False
24153    False
Name: D_TRA1_CASE, Length: 24154, dtype: bool

In [32]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
df=df[(df['D_TRA1_CASE']==2) | (df['D_TRA1_CASE']==0)]
df=df[(df['D_TRA2_CASE']==2) | (df['D_TRA2_CASE']==0)]
df=df[(df['D_TRA3_CASE']==2) | (df['D_TRA3_CASE']==0)]
df=df[(df['D_TRA4_CASE']==2) | (df['D_TRA4_CASE']==0)]

In [33]:
df.groupby('D_TRA1_CASE').count()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_SYEAR,D_TRA1_SMONTH,D_TRA1_SDAY,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
D_TRA1_CASE,,,,,,,,,,,,,,,,,,,,,
0.0,9778,9778,9778,9778,756,756,756,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,10813,10813,10813,10813,936,936,936,10813,10813,10813,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# 여행 유형이 "2.국내여행"인 레코드만 추출
# df=df[df['D_TRA1_CASE']==2 | df['D_TRA2_CASE']==2]
# df=df[(df['D_TRA2_CASE']==2)]
# df=df[df['D_TRA3_CASE']==2]
# df=df[df['D_TRA4_CASE']==2]

In [35]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A12']>=3) & (df['A13']>=3)]

In [36]:
## 1차 필요컬럼
columns_list = [
#   ============================ 재방문 의도, 타인추천 의도
    'A13',
    'A14',
#   ============================ 여행지에서의 활동
    'A3_1',
    'A3_2',
    'A3_3',
    'A3_4',
    'A3_5',
    'A3_6',
    'A3_7',
    'A3_8',
    'A3_9',
    'A3_10',
    'A3_11',
    'A3_12',
    'A3_12',
    'A3_13',
    'A3_14',
    'A3_15',
    'A3_16',
    'A3_17',
    'A3_18',
    'A3_19',
    'A3_20',
    'A3_21',
#   ============================ 여행 시작 날짜, 종료 날짜, 방문 지역, 각 방문 지역의 시작, 종료 날짜
    'D_TRA1_SYEAR', 
    'D_TRA1_SMONTH',
    'D_TRA1_SDAY',
    'D_TRA1_EYEAR',
    'D_TRA1_EMONTH',
    'D_TRA1_EDAY',
    
    'D_TRA1_1_CITY1',
    'D_TRA1_1_TOWN1',
    'D_TRA1_1_CITY2',
    'D_TRA1_1_TOWN2',
    'D_TRA1_1_CITY3',
    'D_TRA1_1_TOWN3',
    'D_TRA1_1_SYEAR',
    'D_TRA1_1_SMONTH',
    'D_TRA1_1_SDAY',
    'D_TRA1_1_EYEAR',
    'D_TRA1_1_EMONTH',
    'D_TRA1_1_EDAY',
    'D_TRA1_2_CITY1',
    'D_TRA1_2_TOWN1',
    'D_TRA1_2_SYEAR',
    'D_TRA1_2_SMONTH',
    'D_TRA1_2_SDAY',
    'D_TRA1_2_EYEAR',
    'D_TRA1_2_EMONTH',
    'D_TRA1_2_EDAY',
#   ============================  
    'D_TRA2_SYEAR', 
    'D_TRA2_SMONTH',
    'D_TRA2_SDAY',
    'D_TRA2_EYEAR',
    'D_TRA2_EMONTH',
    'D_TRA2_EDAY',
    
    'D_TRA2_1_CITY1',
    'D_TRA2_1_TOWN1',
    'D_TRA2_1_CITY2',
    'D_TRA2_1_TOWN2',
    'D_TRA2_1_CITY3',
    'D_TRA2_1_TOWN3',
    'D_TRA2_1_SYEAR',
    'D_TRA2_1_SMONTH',
    'D_TRA2_1_SDAY',
    'D_TRA2_1_EYEAR',
    'D_TRA2_1_EMONTH',
    'D_TRA2_1_EDAY',
    'D_TRA2_2_CITY1',
    'D_TRA2_2_TOWN1',
    'D_TRA2_2_SYEAR',
    'D_TRA2_2_SMONTH',
    'D_TRA2_2_SDAY',
    'D_TRA2_2_EYEAR',
    'D_TRA2_2_EMONTH',
    'D_TRA2_2_EDAY',
#   ============================    
    'D_TRA3_SYEAR', 
    'D_TRA3_SMONTH',
    'D_TRA3_SDAY',
    'D_TRA3_EYEAR',
    'D_TRA3_EMONTH',
    'D_TRA3_EDAY',
    
    'D_TRA3_1_CITY1',
    'D_TRA3_1_TOWN1',
    'D_TRA3_1_CITY2',
    'D_TRA3_1_TOWN2',
    'D_TRA3_1_CITY3',
    'D_TRA3_1_TOWN3',
    'D_TRA3_1_SYEAR',
    'D_TRA3_1_SMONTH',
    'D_TRA3_1_SDAY',
    'D_TRA3_1_EYEAR',
    'D_TRA3_1_EMONTH',
    'D_TRA3_1_EDAY',
    'D_TRA3_2_CITY1',
    'D_TRA3_2_TOWN1',
    'D_TRA3_2_SYEAR',
    'D_TRA3_2_SMONTH',
    'D_TRA3_2_SDAY',
    'D_TRA3_2_EYEAR',
    'D_TRA3_2_EMONTH',
    'D_TRA3_2_EDAY',
#   ============================    
    'D_TRA4_SYEAR', 
    'D_TRA4_SMONTH',
    'D_TRA4_SDAY',
    'D_TRA4_EYEAR',
    'D_TRA4_EMONTH',
    'D_TRA4_EDAY',
    
    'D_TRA4_1_CITY1',
    'D_TRA4_1_TOWN1',
    'D_TRA4_1_CITY2',
    'D_TRA4_1_TOWN2',
    'D_TRA4_1_CITY3',
    'D_TRA4_1_TOWN3',
    'D_TRA4_1_SYEAR',
    'D_TRA4_1_SMONTH',
    'D_TRA4_1_SDAY',
    'D_TRA4_1_EYEAR',
    'D_TRA4_1_EMONTH',
    'D_TRA4_1_EDAY',
    'D_TRA4_2_CITY1',
    'D_TRA4_2_TOWN1',
    'D_TRA4_2_SYEAR',
    'D_TRA4_2_SMONTH',
    'D_TRA4_2_SDAY',
    'D_TRA4_2_EYEAR',
    'D_TRA4_2_EMONTH',
    'D_TRA4_2_EDAY',
#   ============================  사전예약사항(숙소 등)
    'A8_1',
    'A8_2',
    'A8_3',
    'A8_4',
    'A8_5',
    'A8_6',
    'A8_7',
    'A8_10',
    'A8_11',
#   ============================  주요 이동 교통수단 1~3순위
    'A5_1',
    'A5_2',
    'A5_3',
#   ============================  여행지 선택 이유 1~3순위
    'A2_1',
    'A2_2',
    'A2_3',
#   ============================  동반자 유형
    'A7B_1',
    'A7B_2',
    'A7B_3',
    'A7B_4',
    'A7B_5',
    'A7B_6',
    'A7B_7',
#   ============================  여행 인원수
    'A7',
#   ============================  15세 미만 동만자 수
    'A7A',
#   ============================  여행지 지출 총합
    'N_A10',
#   ============================  전반적 만족도
    'A12',
#   ============================  성별
    'SEX',
#   ============================  나이
    'AGE_1YEAR_UNIT',
#   ============================  출발 월
    'BMON',
#   ============================  거주지
    'BSIDO'
]

In [37]:
# 컬럼명 맞게 들어갔는지 체크
for one in columns_list:
    df[one].head(1)

In [38]:
df.columns

Index(['ID', 'MON_EXP_DOM_TOUR', 'MON_EXP_DOM_BUSIN', 'MON_EXP_DOM_FRI',
       'YEAR_EXP_DOM_TOUR', 'YEAR_EXP_DOM_BUSIN', 'YEAR_EXP_DOM_FRI',
       'D_TRA1_CASE', 'D_TRA1_SYEAR', 'D_TRA1_SMONTH',
       ...
       '국내_여행방문지_기타당일7', '국내_여행방문지_기타당일8', '국내_여행방문지_기타당일9', '국내_여행방문지_기타당일10',
       '국내_여행방문지_기타당일11', '국내_여행방문지_기타당일12', '국내_여행방문지_기타당일13',
       '국내_여행방문지_기타당일14', '국내_여행방문지_기타당일15', '국내_여행방문지_기타당일16'],
      dtype='object', length=1302)

In [39]:
df[columns_list]

,A13,A14,A3_1,A3_2,A3_3,A3_4,A3_5,A3_6,A3_7,A3_8,...,A7B_6,A7B_7,A7,A7A,N_A10,A12,SEX,AGE_1YEAR_UNIT,BMON,BSIDO
3,5.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,360000.000000,4.0,2,62,7,11
5,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,33.0,0.0,57200.000000,4.0,2,53,5,11
6,4.0,4.0,1.0,2.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,5.0,0.0,90000.000000,4.0,1,76,5,11
9,4.0,3.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,68500.000000,4.0,2,66,5,11
10,4.0,4.0,1.0,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,5.0,0.0,42003.514938,4.0,2,65,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24114,3.0,3.0,1.0,NaN,3.0,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,3.0,1.0,45283.018868,4.0,2,46,8,39
24115,4.0,2.0,NaN,2.0,NaN,NaN,NaN,6.0,NaN,NaN,...,NaN,NaN,1.0,NaN,318181.818182,4.0,2,54,8,39
24126,4.0,4.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,5.0,0.0,30000.000000,4.0,2,16,12,39
24132,4.0,4.0,1.0,2.0,NaN,NaN,5.0,6.0,NaN,NaN,...,NaN,NaN,4.0,2.0,127941.176471,4.0,2,26,9,39


In [40]:
df[df["D_TRA1_1_SYEAR"]!=df['D_TRA1_1_EYEAR']].head()

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2877,21010540601_1202813,1,2,2,1.0,2.0,2.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df[df["D_TRA1_SMONTH"]!=df['D_TRA1_EMONTH']]

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
1165,11130700151_1223104,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1176,1113071084A_703602,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,11130762371_1201154,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,1114070066A_101092,1,2,2,NaN,NaN,NaN,2.0,2017.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,11240530281_1202386,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23463,39010120311_724855,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23488,39010121281_1222623,1,2,2,1.0,2.0,2.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23491,39010121361_747628,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23794,3901060089A_748008,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# 여행 박 수 계산을 위한 라이브러리 import
from datetime import datetime

In [43]:
now  = datetime.now()
date_to_compare = datetime.strptime("2022512", "%Y%m%d")
date_diff = now - date_to_compare
print("차이 :", date_diff.days)
type(date_diff.days)

차이 : 1


int

In [44]:
df.loc[6,'D_TRA1_SYEAR']

2018.0

In [45]:
# df['D_TRA1_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA2_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA3_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA4_SYEAR'].fillna(2018, inplace=True)
# df['D_TRA1_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA2_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA3_EYEAR'].fillna(2018, inplace=True)
# df['D_TRA4_EYEAR'].fillna(2018, inplace=True)

In [46]:
df['D_TRA1_1_SYEAR'].isna().sum()

23

In [58]:
df['D_TRA2_1_SYEAR'].isna().sum()

10584

In [59]:
df['D_TRA2_1_SMONTH'].isna().sum()

10584

In [60]:
df['D_TRA2_1_SDAY'].isna().sum()

10584

## 1~4번 여행지 시작일, 종료일 컬럼 각각 추가 (임시 데이터프레임 생성)

In [184]:
date_df=pd.DataFrame()

In [185]:
# 날짜 자료형 object -> int64로 변경

In [186]:
df['D_TRA1_SYEAR'].fillna(-1).astype('int64').replace({-1: np.nan}).astype(str)

3        2018.0
5        2018.0
6        2018.0
9        2018.0
10       2018.0
          ...  
24114    2018.0
24115    2018.0
24126    2018.0
24132    2018.0
24137    2018.0
Name: D_TRA1_SYEAR, Length: 10765, dtype: object

In [187]:
s='안녕하세요'

In [188]:
s[:-2]

'안녕하'

#### 소수점 버리기 함수

In [189]:
def cut_zero(str):
    return str[:-2]

In [190]:
df['D_TRA1_SYEAR'].astype(str).apply(cut_zero)

3        2018
5        2018
6        2018
9        2018
10       2018
         ... 
24114    2018
24115    2018
24126    2018
24132    2018
24137    2018
Name: D_TRA1_SYEAR, Length: 10765, dtype: object

In [191]:
df['D_TRA1_SMONTH'].astype(str).apply(cut_zero)

3         7
5         5
6         5
9         5
10        9
         ..
24114     8
24115     8
24126    12
24132     9
24137     6
Name: D_TRA1_SMONTH, Length: 10765, dtype: object

In [192]:
df['D_TRA1_SDAY'].astype(str).apply(cut_zero)

3         6
5        19
6         8
9         4
10       22
         ..
24114     3
24115    21
24126    25
24132     3
24137     7
Name: D_TRA1_SDAY, Length: 10765, dtype: object

In [193]:
df['D_TRA2_SYEAR']

3       NaN
5       NaN
6       NaN
9       NaN
10      NaN
         ..
24114   NaN
24115   NaN
24126   NaN
24132   NaN
24137   NaN
Name: D_TRA2_SYEAR, Length: 10765, dtype: float64

In [194]:
for i in range(1, 5):
    date_df['start'+str(i)]=df['D_TRA'+str(i)+'_SYEAR'].astype(str).apply(cut_zero)+df['D_TRA'+str(i)+'_SMONTH'].astype(str).apply(cut_zero)+df['D_TRA'+str(i)+'_SDAY'].astype(str).apply(cut_zero)

In [195]:
date_df.head(2)

,start1,start2,start3,start4
3,201876,nnn,nnn,nnn
5,2018519,nnn,nnn,nnn


In [196]:
for i in range(1, 5):
    date_df['end'+str(i)]=df['D_TRA'+str(i)+'_EYEAR'].astype(str).apply(cut_zero)+df['D_TRA'+str(i)+'_EMONTH'].astype(str).apply(cut_zero)+df['D_TRA'+str(i)+'_EDAY'].astype(str).apply(cut_zero)

In [197]:
date_df.head(2)

,start1,start2,start3,start4,end1,end2,end3,end4
3,201876,nnn,nnn,nnn,201878,nnn,nnn,nnn
5,2018519,nnn,nnn,nnn,2018519,nnn,nnn,nnn


In [203]:
date_df['start2'].value_counts()

nnn         10584
20181020       10
20181124        9
20181021        8
20181028        6
            ...  
2018122         1
2018519         1
2018525         1
201866          1
2018113         1
Name: start2, Length: 99, dtype: int64

#### datetime으로 형변환 함수

In [174]:
def toDate(str):
    return datetime.strptime(str, "%Y%m%d")

## datetime으로 날짜 차이 계산하여 박수 컬럼 추가

In [204]:
for i in range(1, 5):
    date_df['D_TRA'+str(i)+'_S_Day']=None

for i in range(0, 24138):
    try:
        differ=toDate(date_df['end1'][i])-toDate(date_df['start1'][i])
        date_df['D_TRA1_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end2'][i])-toDate(date_df['start2'][i])
        date_df['D_TRA2_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end3'][i])-toDate(date_df['start3'][i])
        date_df['D_TRA3_S_Day'][i]=differ.days
    except:
        pass
    
    try:
        differ=toDate(date_df['end4'][i])-toDate(date_df['start4'][i])
        date_df['D_TRA4_S_Day'][i]=differ.days
    except:
        pass

In [206]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10765 entries, 3 to 24137
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   start1        10765 non-null  object
 1   start2        10765 non-null  object
 2   start3        10765 non-null  object
 3   start4        10765 non-null  object
 4   end1          10765 non-null  object
 5   end2          10765 non-null  object
 6   end3          10765 non-null  object
 7   end4          10765 non-null  object
 8   D_TRA1_S_Day  10742 non-null  object
 9   D_TRA2_S_Day  181 non-null    object
 10  D_TRA3_S_Day  17 non-null     object
 11  D_TRA4_S_Day  3 non-null      object
dtypes: object(12)
memory usage: 1.3+ MB


In [207]:
date_df

,start1,start2,start3,start4,end1,end2,end3,end4,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
3,201876,nnn,nnn,nnn,201878,nnn,nnn,nnn,2,None,None,None
5,2018519,nnn,nnn,nnn,2018519,nnn,nnn,nnn,0,None,None,None
6,201858,nnn,nnn,nnn,201858,nnn,nnn,nnn,0,None,None,None
9,201854,nnn,nnn,nnn,201856,nnn,nnn,nnn,2,None,None,None
10,2018922,nnn,nnn,nnn,2018923,nnn,nnn,nnn,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
24114,201883,nnn,nnn,nnn,201884,nnn,nnn,nnn,1,None,None,None
24115,2018821,nnn,nnn,nnn,2018822,nnn,nnn,nnn,1,None,None,None
24126,20181225,nnn,nnn,nnn,20181225,nnn,nnn,nnn,0,None,None,None
24132,201893,nnn,nnn,nnn,201895,nnn,nnn,nnn,2,None,None,None


In [210]:
df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]=date_df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]

In [214]:
df[['D_TRA1_S_Day','D_TRA2_S_Day','D_TRA3_S_Day','D_TRA4_S_Day']]

,D_TRA1_S_Day,D_TRA2_S_Day,D_TRA3_S_Day,D_TRA4_S_Day
3,2,None,None,None
5,0,None,None,None
6,0,None,None,None
9,2,None,None,None
10,1,None,None,None
...,...,...,...,...
24114,1,None,None,None
24115,1,None,None,None
24126,0,None,None,None
24132,2,None,None,None


## ===========================================

In [215]:
## 퍼진 컬럼들 압축 과정

## TRA1_1, TRA1_2 등 모든 여행지 파생변수 area로 ,로 구분해서 입력, 
## 여행지 개수 count하여 no_of_place 컬럼에입력
# 1-1 1-2 1-3 1-4 1-5 ... 8-10
# [1,2,3,4,5,6]
sleepnights_list = [] #여행지에서 잔 밤수(박) ex) 1박 2일 = 1
cost_list = [] # 여행비용 <=이후 중복컬럼 생겨서 미사용
area_list = [] # 거주지 
count_list = [] # 여행지 개수 일치확인용 
month_list = [] # 여행 출발 월
for index in df.index:
    sleepnights = ''
    cost = ''
    area = ''
    count = 0
    month = ''
    # accommodation = ''
    for i in range (1,5):
        col_sleepnights = 'D_TRA'+str(i)+'_S_Day'       
        col_cost = 'D_TRA'+str(i)+'_COST'        
        try: 
            sleepnights = sleepnights + str(int(df.loc[index,col_sleepnights])) + ','
            cost = cost + str(int(df.loc[index,col_cost])) +','
        except: pass
        col_month = 'D_TRA'+str(i)+'_SMONTH'
        try:
                month = month + str(int(df.loc[index, col_month])) + ','
        except:
                pass
        for j in range(1,3):

            col_area ='D_TRA'+str(i)+'_'+str(j)+'_TOWN1'
            
            
            try: 
                area = area + str(int(df.loc[index, col_area])) + ','
                count += 1
            except: pass
            
            # for k in range(1,15):
            #     col_accommodation = 'D_TRA'+str(i)+'_'+str(j)+'A6_'+str(k)
            #     try:
            #         accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
            #     except:
            #         pass
    try:
        area = area.rstrip(area[-1])
    except:pass
    try:
        sleepnights = sleepnights.rstrip(sleepnights[-1])
    except:pass
    try:
        cost = cost.rstrip(cost[-1])
    except:pass
    try:
        month = month.rstrip(month[-1])
    except:pass
    # try:
    #     accommodation = accommodation.rstrip(accommodation[-1])
    # except:pass
    area_list.append(area)
    count_list.append(count)
    month_list.append(month)
    cost_list.append(cost)
    # accommodation_list.append(accommodation)
    sleepnights_list.append(sleepnights)

df['area']= area_list
df['no_of_place'] = count_list
df['month'] = month_list
df['cost'] = cost_list
df['sleepnights'] = sleepnights_list
# df['accommodation'] = accommodation_list

In [219]:
df[['area','no_of_place','month','cost','sleepnights']][df['no_of_place']>1].head(50)

,area,no_of_place,month,cost,sleepnights
195,"39020,25040,25040",3,"5,5,5","500000,150000,397500","3,1,1"
978,"32310,32310",2,"6,6","165000,80000","1,1"
990,"31350,33370",2,"3,3","90000,51000","3,1"
1169,"21090,31240",2,"12,12","320000,70000","2,0"
1176,"32060,31120",2,7,200000,3
1213,"32060,31120",2,8,130000,2
1466,"32390,31370",2,7,150000,2
1573,"38110,37360",2,4,90000,0
1619,"31370,33380",2,"3,3","81000,100000","0,0"
2680,"32410,32410,32410",3,"12,12,12","150000,60000,150000","0,0,0"


In [220]:
accommodation_list = []
for index in df.index:
    accommodation = ''
    for k in range(1,12):
        col_accommodation = 'A8_'+str(k)
        try:
            accommodation = accommodation + str(int(df.loc[index, col_accommodation])) + ','
        except:
            pass
    try:
        accommodation = accommodation.rstrip(accommodation[-1])
    except:pass
    accommodation_list.append(accommodation)
df['accommodation'] = accommodation_list

In [223]:
df[['accommodation', 'sleepnights']].head(50)

,accommodation,sleepnights
3,"1,3",2
5,"2,3,4",0
6,11,0
9,11,2
10,1,1
14,11,0
16,11,1
18,1,1
19,1,1
23,6,0


In [ ]:
## 타겟 컬럼 list (미사용포함)
target_rawcol = []
for i in range(1,22):
    target_rawcol.append("A3_"+str(i))
target_rawcol